<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/to_hub_swiss_classificationn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers 
from datasets import load_dataset
data = load_dataset("swiss_judgment_prediction", "de")

Generating validation split:   0%|          | 0/4705 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9725 [00:00<?, ? examples/s]

Dataset swiss_judgment_prediction downloaded and prepared to /root/.cache/huggingface/datasets/swiss_judgment_prediction/de/2.0.0/f5085addd5983c34508f8eadfccca6542995eddea38acf13fbccb06952988155. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [3]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback,
    Trainer
)

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

In [9]:
tokenized_data = data.map(preprocess_function, batched=True)

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=2)

In [10]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [23]:
def compute_metrics(eval_pred):
    metric1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    micro1 = metric1.compute(predictions=predictions, references=labels, average="micro")["f1"]
    macro1 = metric1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return { "f1-micro": micro1, "f1-macro": macro1}

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True
)

In [26]:
from transformers import TrainerCallback

In [27]:
class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

In [ ]:
trainer = Trainer(
    model=model,
    compute_metrics = compute_metrics,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
)
trainer.add_callback(CustomCallback(trainer)) 
trainer.train()

/content/./results is already a clone of https://huggingface.co/danielsaggau/results. Make sure you pull the latest changes with `repo.git_pull()`.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: canton, id, source_language, legal area, year, language, region, text. If canton, id, source_language, legal area, year, language, region, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 35458
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distrib

Step,Training Loss
500,0.547400
1000,0.534800


In [ ]:
trainer.push_to_hub()